In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import networkx as nx
import freeman as fm

In [2]:
g = fm.load('marvel.gml')
g.label_nodes('name')
g.set_all_nodes(size=15, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.5))

In [3]:
bc = nx.betweenness_centrality(g)

In [4]:
import pandas as pd
data = pd.DataFrame({
    'id' : [i for i in g.nodes],
    'Name': [g.nodes[i]['name'] for i in g.nodes],
    'ConnAmnt': [g.nodes[i]['amount'] for i in g.nodes],
    'Intermediation (r)': [i for i in bc.values()],
})

In [5]:
char_infos = pd.read_csv('../dataset/characters_stats.csv', index_col=0)
semi_df = pd.merge(data, char_infos, how='inner', on=['Name']) 

In [6]:
import numpy as np
semi_df = semi_df.replace('nan', np.nan)
semi_df = semi_df[semi_df['ConnAmnt'].notna()]
semi_df['ConnAmnt'] = pd.to_numeric(semi_df['ConnAmnt'])

In [7]:
semi_df = semi_df.drop_duplicates(subset=['Name'], keep='first')
semi_df = semi_df[semi_df['Alignment'] != 'neutral']
semi_df = semi_df.replace(['good', 'bad'], [0, 1])

In [8]:
semi_df['Intelligence (%)'] = semi_df['Intelligence']/semi_df['Total']
semi_df['Power (%)'] = semi_df['Power']/semi_df['Total']
semi_df['Strength (%)'] = semi_df['Strength']/semi_df['Total']
semi_df['Combat (%)'] = semi_df['Combat']/semi_df['Total']

In [9]:
import statsmodels.api as sm

# model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Strength (%)', 
#                                                        'Power', 'Power (%)', 'Alignment']])
model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Power', 'Alignment']])
result = model.fit()
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Intermediation (r)   R-squared (uncentered):                   0.453
Model:                            OLS   Adj. R-squared (uncentered):              0.438
Method:                 Least Squares   F-statistic:                              29.62
Date:                Mon, 23 Nov 2020   Prob (F-statistic):                    6.07e-18
Time:                        10:39:28   Log-Likelihood:                          601.67
No. Observations:                 147   AIC:                                     -1195.
Df Residuals:                     143   BIC:                                     -1183.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intelligence  7.733e-05   1.49e-05      5.175      0.000    4.78e-05       0.000
Strength      2.241e-06   1.21e-05      0.186      0.853   -2.16e-05    2.61e-05
Power        -1.747e-05   1.53e-05     -1.140      0.256   -4.78e-05    1.28e-05
Alignment       -0.0025      0.001     -3.233      0.002      -0.004      -0.001
==============================================================================
Omnibus:                       51.394   Durbin-Watson:                   1.440
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              109.049
Skew:                           1.544   Prob(JB):                     2.09e-24
Kurtosis:                       5.875   Cond. No.                         225.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
print(f"Coef:\n{result.params}\n")
print(f"std err:\n{result.bse}\n")
print(f"p-values:\n{result.pvalues}\n")

Coef:
Intelligence    0.000077
Strength        0.000002
Power          -0.000017
Alignment      -0.002545
dtype: float64

std err:
Intelligence    0.000015
Strength        0.000012
Power           0.000015
Alignment       0.000787
dtype: float64

p-values:
Intelligence    7.561525e-07
Strength        8.528185e-01
Power           2.563619e-01
Alignment       1.521462e-03
dtype: float64



In [11]:
# g = fm.load('marvelWeights.gml')
# g.label_nodes('name')
# g.set_all_nodes(size=15, labpos='hover')
# g.set_all_edges(color=(0, 0, 0, 0.5))

# intelligence = {}

# for i in g.nodes:
#     intelligence[i] = float(g.nodes[i]['intelligence'])

# g.scale_nodes_size(intelligence)
# g.draw()